In [1]:
%reload_ext dotenv
%dotenv

In [6]:
from tqdm.notebook import tqdm
import pandas as pd
from typing import Optional, List, Tuple
import matplotlib.pyplot as plt
from datasets import Dataset, load_dataset

from langchain.docstore.document import Document as LangchainDocument
from langchain.text_splitter import RecursiveCharacterTextSplitter

pd.set_option('display.max_colwidth', None)

In [3]:
ds = load_dataset("m-ric/huggingface_doc", split='train')

In [5]:
RAW_KB = [ LangchainDocument(page_content=doc['text'], metadata={'source': doc['source']}) for doc in tqdm(ds)]

  0%|          | 0/2647 [00:00<?, ?it/s]

#### Recursive Chunking

- Breakdown the text using a list of separators sorted from the most imp to least
- If first sep doesnt provide the right chunk size, go to next and so on

In [7]:
MARDOWN_SEP = [
                "\n#{1,6} ",
                "```\n",
                "\n\\*\\*\\*+\n",
                "\n---+\n",
                "\n___+\n",
                "\n\n",
                "\n",
                " ",
                ""
            ]